In [120]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
import time
import copy
import numpy as np
import tqdm
import joblib
import os

import pytest

from metadetect.lsst.metadetect import run_metadetect
import descwl_shear_sims as sim

In [122]:
CONFIG = {
    "meas_type": "wmom",
    "metacal": {
        "use_noise_image": True,
        "psf": "fitgauss",
    },
    "psf": {
        "model": "gauss",
        "lm_pars": {},
        "ntry": 2,
    },
    "weight": {
        "fwhm": 1.2,
    },
    "detect": {
        "thresh": 10.0,
    },
}

In [123]:
def _shear_cuts(arr):
    assert arr is not None
    msk = (
        (arr['wmom_flags'] == 0)
        & (arr['wmom_s2n'] > 10)
        & (arr['wmom_T_ratio'] > 1.2)
    )
    return msk


def _meas_shear_data(res):
    msk = _shear_cuts(res['noshear'])
    g1 = np.mean(res['noshear']['wmom_g'][msk, 0])
    g2 = np.mean(res['noshear']['wmom_g'][msk, 1])

    msk = _shear_cuts(res['1p'])
    g1_1p = np.mean(res['1p']['wmom_g'][msk, 0])
    msk = _shear_cuts(res['1m'])
    g1_1m = np.mean(res['1m']['wmom_g'][msk, 0])
    R11 = (g1_1p - g1_1m) / 0.02

    dt = [
        ('g1', 'f8'),
        ('g2', 'f8'),
        ('R11', 'f8'),
    ]
    return np.array([(g1, g2, R11)], dtype=dt)


def _bootstrap_stat(d1, d2, func, seed, nboot=500):
    dim = d1.shape[0]
    rng = np.random.RandomState(seed=seed)
    stats = []
    for _ in tqdm.trange(nboot, leave=False):
        ind = rng.choice(dim, size=dim, replace=True)
        stats.append(func(d1[ind], d2[ind]))
    return stats


def _meas_m_c_cancel(pres, mres):
    x = np.mean(pres['g1'] - mres['g1'])/2
    y = np.mean(pres['R11'] + mres['R11'])/2
    m = x/y/0.02 - 1

    x = np.mean(pres['g2'] + mres['g2'])/2
    y = np.mean(pres['R11'] + mres['R11'])/2
    c = x/y

    return m, c


def _boostrap_m_c(pres, mres):
    m, c = _meas_m_c_cancel(pres, mres)
    bdata = _bootstrap_stat(pres, mres, _meas_m_c_cancel, 14324, nboot=500)
    merr, cerr = np.std(bdata, axis=0)
    return m, merr, c, cerr


In [124]:
!export CATSIM_DIR=/path/to/catsim
from xlens.simulation.simulator.base import SimulateImageHalo
import astropy.table as astable
import fitsio

In [125]:
# Prepare a halo
src_halo = astable.Table()
src_halo["index"] = np.array([1, 2])
src_halo["mass"] = np.array([4e14, 8e14])
src_halo["conc"] = np.array([6.0, 4.0])
src_halo["z_lens"] = np.array([0.2, 0.52])

In [126]:
simulator = SimulateImageHalo("/global/cfs/cdirs/des/zhou/cluster_shear/repos/xlens/examples/cluster/config.ini")

with open("/global/cfs/cdirs/des/zhou/cluster_shear/repos/xlens/examples/cluster/config.ini", "r") as f:
    contents = f.read()
    print(contents)

import os 
os.environ['CATSIM_DIR'] = '/global/cfs/cdirs/des/zhou/cluster_shear/data/catsim'

simulator.run(ifield=1, src_halo=src_halo[1])

[simulation]
root_dir    =   ./
# image directory name
sim_name    =   sim
# catalog directory name
cat_dir     =   test
sum_dir     =   test

# layout
layout = random_disk
# number of rotation
nrot = 2
# number of pixels
coadd_dim = 500
# buff on each side
buff = 20

rotate = False
dither = False

draw_bright = False
star_bleeds = False
cosmic_rays = False
bad_columns = False

psf_variation = 0.0
stellar_density = 0.0
survey_name = LSST

band        =   r
noise_ratio =   1.0
psf_fwhm    =   0.8

Simulating for field: 1, and halo index 2
galsim.Moffat(beta=2.5, scale_radius=0.7076510959648599).transform(1.0,0.0,0.0,1.0)
Simulation has galaxies: 435
1.0
1.0


In [127]:
sim_data_list = simulator.sim_data_list
assert sim_data_list[0]['band_data']['i'][0].getPsf() is not None
print(len(sim_data_list))

2


In [128]:
exp = sim_data_list[0]['band_data']['i'][0]
sim_data = sim_data_list[0]

In [129]:
exp.getVariance()

lsst.afw.image._image.ImageF=[[0.35490522 0.35490522 0.35490522 ... 0.35490522 0.35490522 0.35490522]
 [0.35490522 0.35490522 0.35490522 ... 0.35490522 0.35490522 0.35490522]
 [0.35490522 0.35490522 0.35490522 ... 0.35490522 0.35490522 0.35490522]
 ...
 [0.35490522 0.35490522 0.35490522 ... 0.35490522 0.35490522 0.35490522]
 [0.35490522 0.35490522 0.35490522 ... 0.35490522 0.35490522 0.35490522]
 [0.35490522 0.35490522 0.35490522 ... 0.35490522 0.35490522 0.35490522]], bbox=(minimum=(0, 0), maximum=(509, 509))

In [130]:
from descwl_coadd.coadd_nowarp import make_coadd_nowarp

In [131]:
from metadetect.lsst.util import extract_multiband_coadd_data
raw_coadd_data = [make_coadd_nowarp(
    exp=exp,
    psf_dims=sim_data['psf_dims'],
    rng = np.random.RandomState(seed=116),
    remove_poisson=False,
)]

coadd_data = extract_multiband_coadd_data(raw_coadd_data)

In [132]:
mdet_rng = np.random.RandomState(seed=1234)
results = run_metadetect(
    rng=mdet_rng,
    config = copy.deepcopy(CONFIG),
    **coadd_data,
)

Exception in base_SkyCoord.measure on record 1: 
  File "src/table/Schema.cc", line 103, in lsst::afw::table::SchemaItem<T> lsst::afw::table::detail::SchemaImpl::find(const std::string&) const [with T = float; std::string = std::__cxx11::basic_string<char>]
    Field with name 'coord_raErr' not found with type 'F'. {0}
lsst::pex::exceptions::NotFoundError: 'Field with name 'coord_raErr' not found with type 'F'.'

Exception in base_SkyCoord.measure on record 2: 
  File "src/table/Schema.cc", line 103, in lsst::afw::table::SchemaItem<T> lsst::afw::table::detail::SchemaImpl::find(const std::string&) const [with T = float; std::string = std::__cxx11::basic_string<char>]
    Field with name 'coord_raErr' not found with type 'F'. {0}
lsst::pex::exceptions::NotFoundError: 'Field with name 'coord_raErr' not found with type 'F'.'

Exception in base_SkyCoord.measure on record 3: 
  File "src/table/Schema.cc", line 103, in lsst::afw::table::SchemaItem<T> lsst::afw::table::detail::SchemaImpl::find

In [133]:
results

{'noshear': array([(1048576, 1, [            nan,             nan],        nan, 1,          nan, [            nan,             nan], [[           nan,            nan], [           nan,            nan]],        nan, 1,        nan,        nan, 1,           nan,        nan, 'nan', 32, 0, 0,   5.785713 , 125.462425 , -0.21428688,  0.46242377, 200.03489098, -0.07209521, 0, [-3.13214353e-06, -3.11286725e-06], 0.32224579, 0, 0, 0.),
        (1048576, 1, [            nan,             nan],        nan, 1,          nan, [            nan,             nan], [[           nan,            nan], [           nan,            nan]],        nan, 1,        nan,        nan, 1,           nan,        nan, 'nan', 32, 0, 0,   5.8968954, 324.74564  , -0.10310446, -0.25434977, 200.02381968, -0.07208903, 0, [-3.13214353e-06, -3.11286725e-06], 0.32224579, 0, 0, 0.),
        (1048576, 1, [            nan,             nan],        nan, 1,          nan, [            nan,             nan], [[           nan,            